# Note

The data used in this project was a few hundred gigabytes in size, which is way too large to be uploaded to github, and so the full data is not provided here. I've provided the code here for demonstrational purposes.

In [1]:
# Import relevant libraries
import os
import h5py
import torch
import numpy as np
import pyvista as pv
import torch.nn as nn
import matplotlib.pyplot as plt
#from icosphere import icosphere
import torch_geometric.nn as gnn
from scipy.spatial import cKDTree
from torch.nn.functional import dropout

# Use GPU as device if it is available, otherwise use CPU
if torch.cuda.is_available():  
    dev = "cuda:0"
    print('Using: {}'.format(torch.cuda.get_device_name(0)))
else:  
    dev = "cpu"
    print('Using CPU')
device = torch.device(dev)

Using: NVIDIA GeForce RTX 3050 Laptop GPU


# Helper Functions

In [2]:
# Pyvista faces are stored differently to other 3D software like GOSPL or Blender (cells)
# So we need to convert between the two types of mesh representations
def facesToCells(faces):
    cells = []
    currentCell = []
    idx = 0
    for i, f in enumerate(faces):
        if idx == 0:
            cells.append(currentCell)
            currentCell = []
            idx = f
        else:
            currentCell.append(f)
            idx -= 1
    return np.array(cells[1:])

# Converts cells format back to pyvista faces
def cellsToFaces(cells):
    faces = []
    for c in cells:
        faces.append(3)
        for x in c:
            faces.append(x)
    return np.array(faces)

# Converts cells to bidirectional edge indices
def getBidirEdgeIndex(cells):
    edges = []
    for cell in cells:
        edges.append([cell[0], cell[1]])
        edges.append([cell[1], cell[2]])
        edges.append([cell[2], cell[1]])
        edges.append([cell[1], cell[0]])
        edges.append([cell[2], cell[0]])
        edges.append([cell[0], cell[2]])
    edges = np.array(edges)
    edges = np.unique(edges, axis=0)
    return edges

# Read and pre-process data from GOSPL
class GosplReader:
    def __init__(self,
                 dataDirFormat = './Data/GosplTrials/Trial{}/NoiseSphere/h5/gospl.{}.p0.h5',
                 unitIcosphereDir = './Data/UnitIcosphere.npz',
                 totalIterations = 60
                ):
        
        # Store parameters
        self.dataDirFormat = dataDirFormat
        self.totalIterations = totalIterations
        
        # Set unit sphere points and faces
        unitIcosphereFile = np.load(unitIcosphereDir)
        self.unitSphereXYZ = unitIcosphereFile['XYZ']
        self.unitSphereFaces = unitIcosphereFile['faces']
        self.cells = facesToCells(self.unitSphereFaces)
        self.edges = getBidirEdgeIndex(self.cells)
        self.unitSphereMesh =  pv.PolyData(self.unitSphereXYZ, self.unitSphereFaces)
    
    # Read HDF5 files generated by GOSPL
    def readHDF5File(self, fileDir):
        gosplDict = {}
        with h5py.File(fileDir, "r") as f:
            for key in f.keys():
                gosplDict[key] = np.array(f[key])
        return gosplDict
    
    # Get data from first and last iteration of GOSPL simulation
    def getTrial(self, trialNum):
        
        # Get data from first GOSPL iteration, which will be the input of our model
        dataDir = self.dataDirFormat.format(trialNum, 0)
        gosplDict = self.readHDF5File(dataDir)
        initElev = gosplDict['elev']
        
        # Get final data from GOSPL, the target data for our model
        dataDir = self.dataDirFormat.format(trialNum, self.totalIterations)
        gosplDict = self.readHDF5File(dataDir)
        finalElev = gosplDict['elev']
        uplift = gosplDict['uplift']
        erodep = gosplDict['erodep']
        flowAcc = np.log(gosplDict['flowAcc'])
        
        # Pack our data into tensors
        feat = np.concatenate((self.unitSphereXYZ, initElev, uplift), axis=1)
        targ = np.concatenate((finalElev, erodep, flowAcc), axis=1)
        data = {
            'features' : feat,
            'targets' : targ,
            'edges' : self.edges}
        return data

# Creates a new folder in which we will save our results in
def createModelsDir(modelsMainDir='./Models', trialDirFormat='{}/IcoUnet{}EdgeFix'):
    trialNumber = 0
    trialDir = trialDirFormat.format(modelsMainDir, trialNumber)
    while os.path.isdir(trialDir):
        trialNumber += 1
        trialDir = trialDirFormat.format(modelsMainDir, trialNumber)
    progressImageDir = trialDir + '/ProgressImages'
    os.mkdir(trialDir)
    os.mkdir(progressImageDir)
    return progressImageDir, trialDir

# Tool to dynamically plot and save loss data during training
class LossPlotter:
    def __init__(self, 
                 updateEveryNepochs=200, 
                 nLossesToStore=20000,
                 mainDir = './Models/IcoUnet2',
                 imageDirFormat = '/LossPlot{}.png',
                 lossTextFile = '/Loss.txt'
                ):
        
        %matplotlib notebook
        
        # Save parameters
        self.updateEveryNepochs = updateEveryNepochs
        self.nLossesToStore = nLossesToStore
        self.losses = np.zeros(nLossesToStore)
        self.mainDir = mainDir
        self.imageDirFormat = mainDir + imageDirFormat
        self.lossTextFile = mainDir + lossTextFile
        
        # Set up figure
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111)
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
        
        # Create new text file
        textWriter = open(self.lossTextFile, "x")
        textWriter.close()
    
    def drawPlot(self, epoch):
        epochs = np.arange(epoch - self.nLossesToStore, epoch)
        self.ax.clear()
        self.ax.plot(epochs, self.losses)
        self.ax.set_xlabel('Epoch')
        self.ax.set_ylabel('Loss')
        self.fig.canvas.draw()
    
    def updatePlotter(self, loss, epoch):
        idx = (epoch % self.updateEveryNepochs) + self.nLossesToStore - self.updateEveryNepochs
        self.losses[idx] = loss
        if epoch%self.updateEveryNepochs == 0:
            self.drawPlot(epoch)
            self.losses = np.roll(self.losses, -self.updateEveryNepochs)
            if epoch%self.nLossesToStore == 0:
                #self.fig.savefig(self.imageDirFormat.format(epoch))
                textWriter = open(self.lossTextFile, "a")
                for l in self.losses:
                    textWriter.write('{},'.format(l))
                textWriter.close()
                
# Example code for reading a single training feature/target pair
gospRead = GosplReader()
trialDat = gospRead.getTrial(0)
features = trialDat['features']
targets = trialDat['targets']

# Print shape of data
print(features.shape)
print(targets.shape)

(40962, 5)
(40962, 3)


# Icosphere Files

These files contain mesh data for the icospheres that we will be using. They were originally created with the stripy library, which was incompatible with this version of python (which I need for GNN), so I saved them as to file instead.

In [3]:
# Read NPZ files for Icosphere data
def getIcosphere(subdivisions):
    fileDir = './Data/SpheresNPZ/IcosphereSubs{}.npz'.format(subdivisions)
    data = np.load(fileDir)
    return data['vertices'], data['cells']

# Get data
verts, cells = getIcosphere(1)
faces = cellsToFaces(cells)

# Plot an example icosphere
plotter = pv.Plotter()
icoMesh = pv.PolyData(verts, faces)
plotter.add_mesh(icoMesh)
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Plot GOSPL Data Onto Icosphere

Here we show what the target elevations look like as generated by GOSPL.

In [4]:
# Read data
%matplotlib inline
gospRead = GosplReader()
trialDat = gospRead.getTrial(0)
elevations = trialDat['targets'][:, 0]

# Get data
subdivisions = 6
verts, cells = getIcosphere(subdivisions)
faces = cellsToFaces(cells)

# Plot results
plotter = pv.Plotter()
icoMesh = pv.PolyData(verts, faces)
plotter.add_mesh(icoMesh, scalars=elevations[0:verts.shape[0]])
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Plot Bidirectional Edges

The code bellow was used for generating plots used in my report and presentation to help explain the main idea of how the Ico-Unet will work.

In [5]:
# Load data
verts, cells = getIcosphere(1)
edges = getBidirEdgeIndex(cells)
faces = cellsToFaces(cells)

# Create plot
plotter = pv.Plotter()
for edge in edges:
    vert1 = verts[edge[0]]
    vert2 = verts[edge[1]]
    arrow = pv.Arrow(
        start = vert1, 
        direction = (vert2-vert1)/4,
        shaft_radius = 0.02,
        tip_radius = 0.04,
        scale = 0.25)
    plotter.add_mesh(arrow)
sphere = pv.PolyData(verts, faces)
plotter.add_mesh(sphere, color='b')
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Inwards (Pooling) and Outwards (Unpooling) Edges

Creates the list of edges used for pookling and unpooling for our Unet model.

In [7]:
# Get directional edges pointing towards vertices of a lower sub icosphere
def getInEdges(bidirEdges, numInVerts):
    inEdges = []
    for edge in bidirEdges:
        if any(edge < numInVerts):
            if edge[0] > edge[1]:
                inEdges.append(edge)
            else:
                newEdge = [edge[1], edge[0]]
                inEdges.append(newEdge)
    inEdges = np.array(inEdges)
    return np.unique(inEdges, axis=0)

# Get directional edges pointing away from vertices of a lower sub icosphere
def getOutEdges(bidirEdges, numInVerts):
    inEdges = []
    for edge in bidirEdges:
        if any(edge < numInVerts):
            if edge[0] < edge[1]:
                inEdges.append(edge)
            else:
                newEdge = [edge[1], edge[0]]
                inEdges.append(newEdge)
    inEdges = np.array(inEdges)
    return np.unique(inEdges, axis=0)

# Pooling Plots

More plots used in my presentation and report

In [8]:
# Which subdivision icosphere to use
subs = 1

# Get mesh data for current subdivision
verts, cells = getIcosphere(subs)
bidirEdges = getBidirEdgeIndex(cells)
faces = cellsToFaces(cells)

# Get mesh data for lower subdivision
inVerts, inCells = getIcosphere(subs-1)
inFaces = cellsToFaces(inCells)
n = len(inVerts)

# Get inwards or outwards pointing edges
inEdges = getInEdges(bidirEdges, n)
#inEdges = getOutEdges(bidirEdges, n)

# Plot results
plotter = pv.Plotter()
for edge in inEdges:
    vert1 = verts[edge[0]]
    vert2 = verts[edge[1]]
    arrow = pv.Arrow(
        start = vert1, 
        direction = (vert2-vert1),
        shaft_radius = 0.02,
        tip_radius = 0.06,
        scale = 0.5**(subs+1))
    plotter.add_mesh(arrow)
    sphere = pv.Sphere(
        center=vert1,
        radius=0.02,
        theta_resolution=8, 
        phi_resolution=8)
    plotter.add_mesh(sphere, color='y')
sphere = pv.PolyData(verts, faces)
plotter.add_mesh(sphere, color='b')
inSphere = pv.PolyData(inVerts*1.02, inFaces)
plotter.add_mesh(inSphere, color='c')
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Managing Icospheres

This class prepares and manages all our icosphere related data to be used by our pytorch model

In [9]:
# Class for dealing with vertices and edges for various icosphere subdivisions
# The data will be dictionaries or lists of torch.tensor objects
class vertsAndEdges:
    def __init__(self, maxSubdivisions):
        self.maxSubdivisions = maxSubdivisions
        self.setVertsAndCells()
        self.vertsDict = self.getVertsDict()
        self.vertsLengths = self.getVertLengths()
        self.edgeDic = self.getEdgeDict()
        self.inEdgeDict = self.getInEdgesDict()
        self.outEdgeDict = self.getOutEdgesDict()
     
    # Load data of vertices and cells of all icospheres
    def setVertsAndCells(self):
        self.verts = []
        self.cells = []
        self.bidirEdges = []
        for sub in range(self.maxSubdivisions+1):
            verts, cells = getIcosphere(sub)
            self.verts.append(verts)
            self.cells.append(cells)
            self.bidirEdges.append(getBidirEdgeIndex(cells))
    
    # Dictionary containing vertices for all subdivisions of icosphere
    def getVertsDict(self, dev=device):
        return {i : torch.tensor(v).to(dev) for i, v in enumerate(self.verts)}
    
    # Length of vertices for all subdivisions of icosphere
    def getVertLengths(self):
        return [v.shape[0] for v in self.verts]
    
    # Dictionary of edges for all subdivisions of icosphere
    def getEdgeDict(self, dev=device):
        return {i : torch.tensor(e.T).to(dev) for i, e in enumerate(self.bidirEdges)}
    
    # Get inwards pointing edges
    def getInEdgesDict(self, dev=device):
        inEdgeDict = {}
        numVerts = self.getVertLengths()
        for sub in range(self.maxSubdivisions):
            inEdge = getInEdges(self.bidirEdges[sub+1], numVerts[sub])
            inEdgeDict[sub+1] = torch.tensor(inEdge.T).to(dev)
        return inEdgeDict
    
    # Get outwards pointing edges
    def getOutEdgesDict(self, dev=device):
        outEdgeDict = {}
        numVerts = self.getVertLengths()
        for sub in range(self.maxSubdivisions):
            outEdge = getOutEdges(self.bidirEdges[sub+1], numVerts[sub])
            outEdgeDict[sub+1] = torch.tensor(outEdge.T).to(dev)
        return outEdgeDict

# Test the above code works
vertsEdges = vertsAndEdges(6)
print(vertsEdges.inEdgeDict[6])

tensor([[10242, 10242, 10243,  ..., 40960, 40961, 40961],
        [ 6695,  7736,   285,  ...,  9775,     8,  9777]], device='cuda:0')


# Downconvolution Layer

Here we define what a single downconvolution layer should do

In [10]:
# Define down convolution layer
class DownConvLayer(torch.nn.Module):
    def __init__(self, dims=[128, 128, 128]):
        super().__init__()
        
        # Layers
        self.conv1 = gnn.GCNConv(dims[0], dims[1])
        self.conv2 = gnn.GCNConv(dims[1], dims[1])
        self.conv3 = gnn.GCNConv(dims[1], dims[2])
        
    # Forward pass of the layer 
    def forward(self, x, biEdges, inEdges, numVertsInLowerSub):
        
        # Bidirectional Convolution
        h = self.conv1(x, biEdges)
        h = h.relu()
        h = dropout(h, p=0.5, training=self.training)
        
        # Another bidirectional convolution
        h = self.conv2(h, biEdges)
        h = h.relu()
        h = dropout(h, p=0.5, training=self.training)
        
        # Inwards convolution (pooling)
        hPool = self.conv3(h, inEdges)
        hPool = hPool.relu()
        hPool = hPool[:numVertsInLowerSub]
        #hPool = dropout(hPool, p=0.5, training=self.training)
        return h, hPool

# Test the above code
depth = 4

# Prepare verts and edges
ve = vertsAndEdges(6)
numVerts = ve.vertsLengths[depth-1]
verts = ve.vertsDict[depth].type(torch.float32)
edgeBi = ve.edgeDic[depth].type(torch.int64)
edgeIn = ve.inEdgeDict[depth].type(torch.int64)

# Test Downconvolution layer
convLayer = DownConvLayer(dims=[3, 128, 128]).to(device)
h, hIn = convLayer(verts, edgeBi, edgeIn, numVerts)
h = h.detach().clone()
hPool = hIn.detach().clone()

# Make sure number of output vertices of layer is same as vertices of lower sphere subdivision
print(h.shape)
print(hPool.shape)
print(ve.vertsLengths[4])
print(ve.vertsLengths[3])

torch.Size([2562, 128])
torch.Size([642, 128])
2562
642


# Upconvolution Layer

Here we define what a single upconvolution layer should do

In [11]:
# Define up convolution layer
class UpConvLayer(torch.nn.Module):
    def __init__(self, paddingSize, dims=[128, 128, 128]):
        super().__init__()
        
        # Layers
        self.conv1 = gnn.GCNConv(dims[0], dims[0])
        self.conv2 = gnn.GCNConv(2*dims[0], dims[1])
        self.conv3 = gnn.GCNConv(dims[1], dims[2])
        self.pad = nn.ConstantPad2d((0, 0, 0, paddingSize), 0)
        
    # Forward pass of the layer 
    def forward(self, x, xSkip, biEdges, outEdges):
        
        # Outwards convolution (unpooling)
        h = self.pad(x)
        h = self.conv1(h, outEdges)
        h = h.relu()
        h = dropout(h, p=0.5, training=self.training)
        
        # Convolution layer
        h1 = torch.cat((h, xSkip), 1)
        h1 = self.conv2(h1, biEdges)
        h1 = h1.relu()
        h1 = dropout(h1, p=0.5, training=self.training)
        
        # Another convolution layer
        h1 = self.conv3(h1, biEdges)
        h1 = h1.relu()
        #h1 = dropout(h1, p=0.5, training=self.training)
        return h1

# Test the above code
depth = 3

# Prepare verts and edges
ve = vertsAndEdges(6)
verts = ve.vertsDict[depth].type(torch.float32)
skipVerts = ve.vertsDict[depth+1].type(torch.float32)
edgeBi = ve.edgeDic[depth]
edgeOut = ve.outEdgeDict[depth]
padSize = ve.vertsLengths[depth+1] - ve.vertsLengths[depth]

# Test Downconvolution layer
convLayer = UpConvLayer(padSize, dims=[3, 128, 128]).to(device)
output = convLayer(verts, skipVerts, edgeBi, edgeOut)

print(verts.shape)
print(output.shape)
print(ve.vertsLengths[depth])
print(ve.vertsLengths[depth+1])

torch.Size([642, 3])
torch.Size([2562, 128])
642
2562


# IcoUnet Model

Using the up/down convolution layers defined above, we now build the Unet model architecture.

In [13]:
# Define down convolution layer
class IcoUnet(torch.nn.Module):
    def __init__(self, vertsLengths):
        super().__init__()
        
        dim = 128
        l = vertsLengths
        self.norm = nn.BatchNorm1d(5)
        
        # Down convolution layers
        self.downConv6 = DownConvLayer(dims=[5, dim, dim])
        self.downConv5 = DownConvLayer(dims=[dim, dim, dim])
        self.downConv4 = DownConvLayer(dims=[dim, dim, dim])
        self.downConv3 = DownConvLayer(dims=[dim, dim, dim])
        self.downConv2 = DownConvLayer(dims=[dim, dim, dim])
        self.downConv1 = DownConvLayer(dims=[dim, dim, dim])
        
        # Upcovolution layers
        self.upConv1 = UpConvLayer(l[1] - l[0], dims=[dim, dim, dim])
        self.upConv2 = UpConvLayer(l[2] - l[1], dims=[dim, dim, dim])
        self.upConv3 = UpConvLayer(l[3] - l[2], dims=[dim, dim, dim])
        self.upConv4 = UpConvLayer(l[4] - l[3], dims=[dim, dim, dim])
        self.upConv5 = UpConvLayer(l[5] - l[4], dims=[dim, dim, dim])
        self.upConv6 = UpConvLayer(l[6] - l[5], dims=[dim, dim, dim])
        
        # One final bidirectional convolution
        self.lastConv = gnn.GCNConv(dim, 3)
    
    # Forward pass of IcoUnet model
    def forward(self, x, biEdges, inEdges, outEdges, vertsLengths):
        h = self.norm(x)
        
        # Encoder layers (down convolutions)
        hSkip6, h = self.downConv6(h, biEdges[6], inEdges[6], vertsLengths[5])
        hSkip5, h = self.downConv5(h, biEdges[5], inEdges[5], vertsLengths[4])
        hSkip4, h = self.downConv4(h, biEdges[4], inEdges[4], vertsLengths[3])
        hSkip3, h = self.downConv3(h, biEdges[3], inEdges[3], vertsLengths[2])
        hSkip2, h = self.downConv2(h, biEdges[2], inEdges[2], vertsLengths[1])
        hSkip1, h = self.downConv1(h, biEdges[1], inEdges[1], vertsLengths[0])
        
        # Decoder layers (up convolutions)        
        h = self.upConv1(h, hSkip1, biEdges[1], outEdges[1])
        h = self.upConv2(h, hSkip2, biEdges[2], outEdges[2])
        h = self.upConv3(h, hSkip3, biEdges[3], outEdges[3])
        h = self.upConv4(h, hSkip4, biEdges[4], outEdges[4])
        h = self.upConv5(h, hSkip5, biEdges[5], outEdges[5])
        h = self.upConv6(h, hSkip6, biEdges[6], outEdges[6])
        h = self.lastConv(h, biEdges[6])
        return h

# Read and prepare data
ve = vertsAndEdges(6)
gospRead = GosplReader()
trialDat = gospRead.getTrial(0)
features = trialDat['features']
targets = trialDat['targets']
feat = torch.tensor(features, dtype=torch.float32).to(device)
lengths = torch.tensor(ve.vertsLengths).to(device)

# Create Unet and pass data
model = IcoUnet(ve.vertsLengths).to(device)
outputTens = model(feat, ve.edgeDic, ve.inEdgeDict, ve.outEdgeDict, lengths)
output = outputTens.cpu().detach().numpy()

# Plot results of the untrained Ico-Unet
verts, cells = getIcosphere(6)
faces = cellsToFaces(cells)
plotter = pv.Plotter()
icoMesh = pv.PolyData(verts, faces)
plotter.add_mesh(icoMesh, scalars=output[:, 0])
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Custom Loss

Our loss is mostly an Mean Square Error Loss, whilst normalizing the data first, since we don't want the model itself to normalize the loss.

In [14]:
class NormalizedMSELoss:
    def __init__(self, targets):
        self.mean = torch.tensor(np.mean(targets, axis=0), dtype=torch.float32).to(device)[None, :]
        self.std = torch.tensor(np.std(targets, axis=0), dtype=torch.float32).to(device)[None, :]
        self.MSE = nn.MSELoss()
    
    def calculateLoss(self, output, target):
        out = self.norm(output)
        targ = self.norm(target)
        loss = self.MSE(out, targ)
        return loss
    
    def norm(self, x):
        return (x - self.mean) / self.std

# Training Loop

Finally, we train our model

In [15]:
def train(model, data, ve):
    features = data['features']
    targets = data['targets']
    #edges = data['edges']

    feat = torch.tensor(features, dtype=torch.float32).to(device)
    targs = torch.tensor(targets, dtype=torch.float32).to(device)
    
    optimizer.zero_grad()
    out = model(feat, ve.edgeDic, ve.inEdgeDict, ve.outEdgeDict, lengths)
    loss = MSENormed.calculateLoss(out, targs)
    loss.backward()
    optimizer.step()
    return model, loss


epochs = 2
maxTrials = 560
%matplotlib notebook


ve = vertsAndEdges(6)
model = IcoUnet(ve.vertsLengths).to(device)

gospRead = GosplReader()
trialDat = gospRead.getTrial(10)
target = trialDat['targets']
MSENormed = NormalizedMSELoss(target)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
progressImageDir, trialDir = createModelsDir(trialDirFormat='{}/IcoUnetImproved{}')

modelDirFormat = trialDir + '/Weights{}.pt'
lossPlot = LossPlotter(updateEveryNepochs=40, nLossesToStore=2000, mainDir=trialDir)

#for epoch in range(epochs):
epoch = 1
while True:
    trialNum = np.random.randint(maxTrials)
    trialDat = gospRead.getTrial(trialNum)
    
    model, loss = train(model, trialDat, ve)
    loss = loss.cpu().detach().numpy()
    lossPlot.updatePlotter(loss, epoch)
    epoch += 1
    
    if epoch%500 == 0:
        modelDir = modelDirFormat.format(epoch)
        torch.save(model.state_dict(), modelDir)

# Plot Output

We then plot the output of our model.

In [ ]:
gospRead = GosplReader()
trialDat = gospRead.getTrial(10)
features = trialDat['features']
targets = trialDat['targets']
edges = trialDat['edges']

feat = torch.tensor(features, dtype=torch.float32).to(device)
edgs =  torch.tensor(edges).to(device)

prediction = model(feat, ve.edgeDic, ve.inEdgeDict, ve.outEdgeDict, lengths)
prediction = prediction.cpu().detach().numpy()

# Plot results
plotter = pv.Plotter()
plotter.add_mesh(gospRead.unitSphereMesh, scalars=prediction[:, 0])
plotter.show()